add volume thresholds and only trade when the underlying isn't going exponential against the spread positions Also only run a naive backtest

In [ ]:
import nest_asyncio
nest_asyncio.apply()

from ib_insync import *
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from statsmodels.tsa.arima.model import ARIMA            # <--- ARIMA
from IPython.display import display

# --- Parameters ---
symbol1 = 'SPY'
symbol2 = symbol1
lastTradeDateOrContractMonth1 = '20250725'
lastTradeDateOrContractMonth2 = lastTradeDateOrContractMonth1
strike1 = 626
strike2 = 616
right1 = 'P'
right2 = right1
barSize = '1 min'
duration = '7 D'
window = 20
num_std = 2
quantity = 1

ib = IB()
await ib.connectAsync('127.0.0.1', 7497, clientId=np.random.randint(1000, 9999))

# --- Define and qualify option contracts ---
option1 = Option(
    symbol=symbol1,
    lastTradeDateOrContractMonth=lastTradeDateOrContractMonth1,
    strike=strike1,
    right=right1,
    exchange='SMART',
    currency='USD'
)
option2 = Option(
    symbol=symbol2,
    lastTradeDateOrContractMonth=lastTradeDateOrContractMonth2,
    strike=strike2,
    right=right2,
    exchange='SMART',
    currency='USD'
)

await ib.qualifyContractsAsync(option1, option2)

bars1 = ib.reqHistoricalData(
    option1, endDateTime='', durationStr=duration, barSizeSetting=barSize,
    whatToShow='TRADES', useRTH=True, formatDate=1, keepUpToDate=True
)
bars2 = ib.reqHistoricalData(
    option2, endDateTime='', durationStr=duration, barSizeSetting=barSize,
    whatToShow='TRADES', useRTH=True, formatDate=1, keepUpToDate=True
)

fig = go.FigureWidget()
fig.add_scatter(name='Spread', line=dict(color='black'))
fig.add_scatter(name='BB Upper', line=dict(color='green', dash='dash'))
fig.add_scatter(name='BB Lower', line=dict(color='red', dash='dash'))
fig.add_scatter(name='BB Middle', line=dict(color='orange', dash='dot'))
fig.add_scatter(name='Buy Signal', mode='markers', marker=dict(symbol='triangle-up', color='green', size=12))
fig.add_scatter(name='Sell Signal', mode='markers', marker=dict(symbol='triangle-down', color='red', size=12))
fig.update_layout(
    title='Live Spread with Bollinger Bands and Signals',
    xaxis_title='Time',
    yaxis_title='Spread',
    template='plotly_white',
    width=1000,
    height=500
)
display(fig)

pending_entry = None
last_order = None

def make_combo_contract(opt1, opt2, is_long=True):
    combo = Contract()
    combo.symbol = opt1.symbol
    combo.secType = 'BAG'
    combo.currency = opt1.currency
    combo.exchange = 'SMART'
    combo.comboLegs = [
        ComboLeg(conId=opt1.conId, ratio=1, action='BUY' if is_long else 'SELL', exchange='SMART'),
        ComboLeg(conId=opt2.conId, ratio=1, action='SELL' if is_long else 'BUY', exchange='SMART')
    ]
    return combo

def calc_midprice():
    price1 = bars1[-1].close
    price2 = bars2[-1].close
    return round(price1 - price2 +0.01, 2)

def make_combo_order(action, qty, limit_price):
    order = Order(
        action=action, orderType='LMT',
        totalQuantity=qty, lmtPrice=limit_price,
        tif='GTC',
        smartComboRoutingParams=[TagValue('NonGuaranteed', '1')]
    )
    return order

def cancel_existing_order(order):
    if order is not None and order.orderId is not None:
        try:
            ib.cancelOrder(order)
            print(f"Cancelled outstanding order (orderId={order.orderId}).")
        except Exception as e:
            print(f"Error cancelling order: {e}")

def update_chart_and_logic():
    global pending_entry, last_order
    df1 = util.df(bars1).set_index('date').rename(columns={'close': 'Close1'})
    df2 = util.df(bars2).set_index('date').rename(columns={'close': 'Close2'})
    # --- Set frequency to 'min' to remove ARIMA warnings ---
    df1.index = pd.to_datetime(df1.index)
    df2.index = pd.to_datetime(df2.index)
    df1 = df1[~df1.index.duplicated()]
    df2 = df2[~df2.index.duplicated()]
    df1 = df1.asfreq('min')  # minutely
    df2 = df2.asfreq('min')  # minutely

    datafix = df1.join(df2, how='inner', lsuffix='_1', rsuffix='_2')
    datafix['spread'] = datafix['Close1'] - datafix['Close2']
    datafix['bb_middle'] = datafix['spread'].rolling(window).mean()
    datafix['bb_std'] = datafix['spread'].rolling(window).std()
    datafix['bb_upper'] = datafix['bb_middle'] + num_std * datafix['bb_std']
    datafix['bb_lower'] = datafix['bb_middle'] - num_std * datafix['bb_std']

    datafix['prev_spread'] = datafix['spread'].shift(1)
    datafix['prev_bb_lower'] = datafix['bb_lower'].shift(1)
    datafix['prev_bb_upper'] = datafix['bb_upper'].shift(1)
    datafix['bb_signal'] = 'hold'
    buy_mask = (datafix['prev_spread'] > datafix['prev_bb_lower']) & (datafix['spread'] < datafix['bb_lower'])
    sell_mask = (datafix['prev_spread'] < datafix['prev_bb_upper']) & (datafix['spread'] > datafix['bb_upper'])
    datafix.loc[buy_mask, 'bb_signal'] = 'buy'
    datafix.loc[sell_mask, 'bb_signal'] = 'sell'

    ### --- ARIMA filter, only allow signals if ARIMA agrees ---
    arima_window = 80
    arima_order = (1, 0, 0)
    arima_preds = [np.nan] * len(datafix)
    spreads = datafix['spread'].values
    for i in range(arima_window, len(datafix)):
        try:
            window_spread = spreads[i-arima_window:i]
            model = ARIMA(window_spread, order=arima_order)
            fit = model.fit()
            pred = fit.forecast(steps=1)[0]
            arima_preds[i] = pred
        except Exception:
            arima_preds[i] = np.nan
    datafix['arima_pred'] = arima_preds
    datafix['arima_dir'] = np.sign(datafix['arima_pred'] - datafix['spread'].shift(1))

    def filtered_signal(row):
        if row['bb_signal'] == 'buy' and row['arima_dir'] == 1:
            return 'buy'
        elif row['bb_signal'] == 'sell' and row['arima_dir'] == -1:
            return 'sell'
        else:
            return 'hold'
    datafix['filtered_signal'] = datafix.apply(filtered_signal, axis=1)
    ### -- End ARIMA filtering logic ---

    if not datafix.empty:
        last_row = datafix.iloc[-1]
        signal = last_row['filtered_signal']   # <--- Use filtered signal!
        print(f"New or initial bar at {last_row.name} | Spread: {last_row['spread']:.2f} | Signal: {signal}")

        positions = {p.contract.conId: p.position for p in ib.positions()}
        pos1 = positions.get(option1.conId, 0)
        pos2 = positions.get(option2.conId, 0)
        if pos1 == 1 and pos2 == -1:
            current_position = 'long'
        elif pos1 == -1 and pos2 == 1:
            current_position = 'short'
        else:
            current_position = 'flat'

        # --- CANCEL existing outstanding order if signal has flipped
        if last_order is not None:
            if (signal == 'buy' and last_order.action.upper() == 'SELL') or \
               (signal == 'sell' and last_order.action.upper() == 'BUY'):
                cancel_existing_order(last_order)
                last_order = None

        mid = calc_midprice()
        new_order = None
        if current_position == 'long' and signal == 'sell':
            combo = make_combo_contract(option1, option2, is_long=False)
            new_order = make_combo_order('SELL', quantity, mid)
            ib.placeOrder(combo, new_order)
            pending_entry = 'short'
        elif current_position == 'short' and signal == 'buy':
            combo = make_combo_contract(option1, option2, is_long=True)
            new_order = make_combo_order('BUY', quantity, mid)
            ib.placeOrder(combo, new_order)
            pending_entry = 'long'
        elif current_position == 'flat':
            if pending_entry is not None:
                is_long = pending_entry == 'long'
                combo = make_combo_contract(option1, option2, is_long)
                action = 'BUY' if is_long else 'SELL'
                new_order = make_combo_order(action, quantity, mid)
                ib.placeOrder(combo, new_order)
                pending_entry = None
            else:
                if signal == 'buy':
                    combo = make_combo_contract(option1, option2, is_long=True)
                    new_order = make_combo_order('BUY', quantity, mid)
                    ib.placeOrder(combo, new_order)
                elif signal == 'sell':
                    combo = make_combo_contract(option1, option2, is_long=False)
                    new_order = make_combo_order('SELL', quantity, mid)
                    ib.placeOrder(combo, new_order)
        if new_order:
            last_order = new_order

    # --- Update plot (last 780 bars) ---
    plot_df = datafix.tail(780)
    buy_signals = plot_df[plot_df['filtered_signal'] == 'buy']
    sell_signals = plot_df[plot_df['filtered_signal'] == 'sell']
    with fig.batch_update():
        fig.data[0].x = plot_df.index
        fig.data[0].y = plot_df['spread']
        fig.data[1].x = plot_df.index
        fig.data[1].y = plot_df['bb_upper']
        fig.data[2].x = plot_df.index
        fig.data[2].y = plot_df['bb_lower']
        fig.data[3].x = plot_df.index
        fig.data[3].y = plot_df['bb_middle']
        fig.data[4].x = buy_signals.index
        fig.data[4].y = buy_signals['spread']
        fig.data[5].x = sell_signals.index
        fig.data[5].y = sell_signals['spread']

update_chart_and_logic()

def on_new_bar(bars, has_new_bar):
    if has_new_bar:
        update_chart_and_logic()

bars1.updateEvent += on_new_bar
bars2.updateEvent += on_new_bar

# --- Reset last_order if it fills or is fully cancelled ---
def on_order_status(trade):
    global last_order
    status = trade.orderStatus.status
    if status in ('Filled', 'Cancelled'):
        last_order = None

ib.orderStatusEvent += on_order_status

print("Live strategy, trading, and chart are running. Awaiting new bars...")

import asyncio
await asyncio.sleep(1e6)


FigureWidget({
    'data': [{'line': {'color': 'black'},
              'name': 'Spread',
              'type': 'scatter',
              'uid': 'aaed337b-b3c2-4a82-810a-5fe9931a952f'},
             {'line': {'color': 'green', 'dash': 'dash'},
              'name': 'BB Upper',
              'type': 'scatter',
              'uid': '53059edd-b168-4f17-90de-ab107db7f441'},
             {'line': {'color': 'red', 'dash': 'dash'},
              'name': 'BB Lower',
              'type': 'scatter',
              'uid': 'e198c6e3-270c-4e81-83da-f0fb775d24ce'},
             {'line': {'color': 'orange', 'dash': 'dot'},
              'name': 'BB Middle',
              'type': 'scatter',
              'uid': '8208d5df-e7bf-4c07-ac34-4a93961ed2b3'},
             {'marker': {'color': 'green', 'size': 12, 'symbol': 'triangle-up'},
              'mode': 'markers',
              'name': 'Buy Signal',
              'type': 'scatter',
              'uid': '014ebb1a-3c4a-4e25-9608-0a3afd636940'},
           

New or initial bar at 2025-07-15 14:45:00-04:00 | Spread: 3.95 | Signal: hold
Live strategy, trading, and chart are running. Awaiting new bars...
